In [ ]:
!pip install neo4j
!pip3 install tomlkit
!pip3 install -U neo4j
!pip install py2neo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.19.0-py3-none-any.whl size=280741 sha256=efdd8bb3ab7eebb356842325632986ebc0c61dff9d24f36eb02c0dc218fc1837
  Stored in directory: /root/.cache/pip/wheels/30/db/9b/2cfde1fa33145219c0322f299b604daf5aba2ed443a7ed5f07
Successfully built neo4j
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship

df = pd.read_csv('/content/combined_data.csv')
print(df.head())

<ipython-input-10-c1d2bfdfbecd>:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/combined_data.csv')


   rating                                              title parent_asin  \
0     5.0  Tyger Auto T1 Soft Roll Up Truck Bed Tonneau C...  B0BV88374L   
1     5.0  Tailgate Insert Letters Compatible with F150 R...  B0B2WGS5ND   
2     1.0  uxcell Adhesive 3" Round Side Rearview Blind S...  B00A0GV20Q   
3     5.0  Crevelle Custom Fits 2009-2020 Nissan 370Z / 3...  B08C27WWVG   
4     5.0  Pre-Painted Trunk Spoiler Compatible with 2009...  B0719J5ZNY   

                        user_id  average_rating  rating_number  price  \
0  AHMUTK5MAIDB7NQZQYH5NIOPT6BQ             4.8         5239.0  207.0   
1  AFE337D2J37YRU5U6MVTVKNDKWDA             5.0            1.0    NaN   
2  AGMVX6M2O2J7L63ATNZA2KBWHIAQ             3.7            8.0    NaN   
3  AHQOKY3AQJG5WK4QGPCIVRA2UEBQ             4.4           65.0  95.95   
4  AEZBUG7X75XAZ6MSPQ43QWY2WBBQ             4.6           28.0    NaN   

              store                                         categories  
0        Tyger Auto  Automotive

In [24]:
#DATA CLEANING

# Convert 'price' to numeric, converting errors to NaN
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Fill NaN values in 'price' and other necessary columns
df.fillna({
    'rating': 0,
    'title': 'Unknown',
    'parent_asin': 'Unknown',
    'user_id': 'Unknown',
    'average_rating': 0,
    'rating_number': 0,
    'price': 0,  # Default value for NaN prices
    'store': 'Unknown',
    'categories': 'Unknown'
}, inplace=True)

print(df.head())


   rating                                              title parent_asin  \
0     5.0  Tyger Auto T1 Soft Roll Up Truck Bed Tonneau C...  B0BV88374L   
1     5.0  Tailgate Insert Letters Compatible with F150 R...  B0B2WGS5ND   
2     1.0  uxcell Adhesive 3" Round Side Rearview Blind S...  B00A0GV20Q   
3     5.0  Crevelle Custom Fits 2009-2020 Nissan 370Z / 3...  B08C27WWVG   
4     5.0  Pre-Painted Trunk Spoiler Compatible with 2009...  B0719J5ZNY   

                        user_id  average_rating  rating_number   price  \
0  AHMUTK5MAIDB7NQZQYH5NIOPT6BQ             4.8         5239.0  207.00   
1  AFE337D2J37YRU5U6MVTVKNDKWDA             5.0            1.0    0.00   
2  AGMVX6M2O2J7L63ATNZA2KBWHIAQ             3.7            8.0    0.00   
3  AHQOKY3AQJG5WK4QGPCIVRA2UEBQ             4.4           65.0   95.95   
4  AEZBUG7X75XAZ6MSPQ43QWY2WBBQ             4.6           28.0    0.00   

              store                                         categories  
0        Tyger Auto  Auto

In [23]:
print(df.columns)


Index(['rating', 'title', 'parent_asin', 'user_id', 'average_rating',
       'rating_number', 'price', 'store', 'categories'],
      dtype='object')


In [ ]:
# V1 CONNECT TO NEO4J
from neo4j import GraphDatabase


username = "neo4j"
password = "apan5400"
uri = "neo4j:////8.tcp.ngrok.io:17322 -> localhost:7687 "

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

graph = Graph(uri, auth=(username, password))

def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Successfully connected to Neo4j!' AS greeting")
            for record in result:
                print(record['greeting'])
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")
    finally:
        driver.close()

# Test the connection
test_connection()



ERROR:neo4j.pool:Unable to retrieve routing information


Failed to connect to Neo4j: Unable to retrieve routing information


In [ ]:
from py2neo import Graph

graph = Graph("neo4j://8.tcp.ngrok.io:17322", auth=("neo4j", "apan5400"))

# Function to write DataFrame to Neo4j using py2neo
def write_data_to_neo4j(df, graph):
    # Iterate through DataFrame rows
    for index, row in df.iterrows():
        graph.run("""
            MERGE (product:Product {asin: $asin, title: $title, price: $price, average_rating: $average_rating})
            MERGE (customer:Customer {id: $user_id})
            MERGE (customer)-[purchase:PURCHASED {rating: $rating}]->(product)
        """, parameters={
            'asin': row['parent_asin'],
            'title': row['title'],
            'price': row['price'],
            'average_rating': row['average_rating'],
            'user_id': row['user_id'],
            'rating': row['rating']
        })

# Execute the function
write_data_to_neo4j(df, graph)

# Assuming GDS plugin is installed and proper graph projection has been created
centrality_query = """
CALL gds.graph.create(
  'salesGraph',
  ['Product', 'Customer'],
  ['PURCHASED']
)

CALL gds.betweenness.stream('salesGraph')
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
SET n.betweenness = score

CALL gds.pageRank.stream('salesGraph')
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
SET n.pageRank = score

CALL gds.louvain.stream('salesGraph')
YIELD nodeId, communityId
MATCH (n) WHERE id(n) = nodeId
SET n.communityId = communityId
"""
graph.run(centrality_query)

# Retrieve and display the top products based on PageRank
top_products_query = """
MATCH (p:Product)
RETURN p.asin AS ProductASIN, p.pageRank AS PageRank
ORDER BY PageRank DESC
LIMIT 10
"""
top_products = graph.run(top_products_query).to_data_frame()

print("Top Products Based on PageRank:")
print(top_products)

NameError: name 'DatabaseError' is not defined

In [ ]:
#V2
from neo4j import GraphDatabase

uri = "neo4j://.tcp.ngrok.io:17322 -> localhost:7687"
username = "neo4j"
password = "apan5400"

driver = GraphDatabase.driver(uri, auth=(username, password))

def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Successfully connected to Neo4j!' AS greeting")
            for record in result:
                print(record['greeting'])
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")
    finally:
        driver.close()

# Test the connection
test_connection()

In [ ]:
def execute_query(driver, query, parameters=None):
    with driver.session() as session:
        # Run the Cypher query
        result = session.run(query, parameters)
        return result

In [ ]:
#V1
from neo4j import GraphDatabase

cypher_query = '''
UNWIND $products AS product
MERGE (p:Product {asin: product.parent_asin})
  ON CREATE SET p.title = product.title, p.price = product.price
MERGE (u:User {id: product.user_id})
MERGE (u)-[:PURCHASED {rating: product.rating}]->(p)
MERGE (s:Store {name: product.store})
MERGE (p)-[:SOLD_BY]->(s)
WITH product, p
UNWIND product.categories AS category
MERGE (c:Category {name: category})
MERGE (p)-[:BELONGS_TO]->(c)
'''

def execute_query(driver, query, parameters=None):
    # Open a session with the database
    with driver.session() as session:
        # Execute the query in a transaction
        result = session.write_transaction(_execute_tx, query, parameters)
        return result

def _execute_tx(tx, query, parameters):
    # Run the query within the transaction
    result = tx.run(query, parameters)
    return result.summary().counters

# Main execution block
if __name__ == "__main__":
    # Assuming you have a way to create a Neo4j driver
    driver = GraphDatabase.driver("neo4j://0.tcp.ngrok.io:18369 -> localhost:7687", auth=("neo4j", "apan5400"))

    try:
        products = df.to_dict('records')
        execute_query(driver, cypher_query, parameters={'products': products})
        print("Graph creation in Neo4j is complete.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        # Ensure the driver is closed properly
        driver.close()


ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('3.134.125.175', 18369)) (ResolvedIPv4Address(('3.134.125.175', 18369)))
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('0.tcp.ngrok.io', 18369)) (ResolvedIPv4Address(('3.134.125.175', 18369)))
ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('3.14.182.203', 18369)) (ResolvedIPv4Address(('3.14.182.203', 18369)))
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('0.tcp.ngrok.io', 18369)) (ResolvedIPv4Address(('3.14.182.203', 18369)))
<ipython-input-40-f231172bffca>:22: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(_execute_tx, query, parameters)
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve routing information
ERROR:neo4j.pool:Unable to retrieve rout

An error occurred: Unable to retrieve routing information


In [ ]:
# Functions to create nodes and relationships in Neo4j
def create_graph(driver, df):
    # Open a session with the database
    with driver.session() as session:
        # Loop through the DataFrame rows
        for index, row in df.iterrows():
            # Create or merge Product node
            session.run(
                "MERGE (p:Product {asin: $asin}) "
                "ON CREATE SET p.title = $title, p.average_rating = $average_rating, p.price = $price ",
                {'asin': row['parent_asin'], 'title': row['title'],
                 'average_rating': row['average_rating'], 'price': row['price']}
            )

            # Create or merge User node
            session.run(
                "MERGE (u:User {id: $user_id})",
                {'user_id': row['user_id']}
            )

            # Create PURCHASED relationship
            session.run(
                "MATCH (u:User {id: $user_id})"
                "MATCH (p:Product {asin: $asin})"
                "MERGE (u)-[:PURCHASED {rating: $rating}]->(p)",
                {'user_id': row['user_id'], 'asin': row['parent_asin'], 'rating': row['rating']}
            )

            # Split categories and create BELONGS_TO relationships
            categories = row['categories'].split(', ')
            for category_name in categories:
                session.run(
                    "MERGE (c:Category {name: $name})",
                    {'name': category_name}
                )
                session.run(
                    "MATCH (p:Product {asin: $asin})"
                    "MATCH (c:Category {name: $name})"
                    "MERGE (p)-[:BELONGS_TO]->(c)",
                    {'asin': row['parent_asin'], 'name': category_name}
                )

            # Create or merge Store node and SOLD_BY relationship
            session.run(
                "MERGE (s:Store {name: $name})",
                {'name': row['store']}
            )
            session.run(
                "MATCH (p:Product {asin: $asin})"
                "MATCH (s:Store {name: $name})"
                "MERGE (p)-[:SOLD_BY]->(s)",
                {'asin': row['parent_asin'], 'name': row['store']}
            )

# Execute the graph creation
create_graph(driver, df)

# Close the driver connection when done
driver.close()

<ipython-input-62-dc6aeec45bb8>:4: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


KeyboardInterrupt: 

In [ ]:
from neo4j import GraphDatabase
import time

uri = "neo4j://8.tcp.ngrok.io:17322 -> localhost:7687"
username = "neo4j"
password = "apan5400"

driver = GraphDatabase.driver(uri, auth=(username, password))

def test_connection():
    retries = 5
    while retries > 0:
        try:
            with driver.session() as session:
                session.run("RETURN 'Successfully connected to Neo4j!' AS greeting").single()[0]
                print("Successfully connected to Neo4j!")
                return
        except Exception as e:
            print(f"Failed to connect to Neo4j: {e}, retrying in 5 seconds...")
            time.sleep(5)
            retries -= 1
        finally:
            if retries == 0:
                print("All retries failed.")

# Test the connection
test_connection()




Successfully connected to Neo4j!


In [ ]:
#V2
# Function to create nodes and relationships in Neo4j
def create_graph(driver, products, users, categories, stores):
    with driver.session() as session:
        # Add Product nodes
        for product in products:
            session.run("MERGE (p:Product {asin: $asin}) "
                        "SET p.title = $title, p.average_rating = $average_rating, p.price = $price",
                        product)

        # Add User nodes
        for user in users:
            session.run("MERGE (u:User {user_id: $user_id})", user)

        # Add Category nodes
        for category in categories:
            session.run("MERGE (c:Category {name: $name})", category)

        # Add Store nodes
        for store in stores:
            session.run("MERGE (s:Store {name: $name})", store)

        # Add PURCHASED relationships
        for purchase in purchases:
            session.run("MATCH (u:User {user_id: $user_id})"
                        "MATCH (p:Product {asin: $asin})"
                        "MERGE (u)-[:PURCHASED {rating: $rating}]->(p)",
                        purchase)

        # Add BELONGS_TO relationships
        for product_category in product_categories:
            session.run("MATCH (p:Product {asin: $asin})"
                        "MATCH (c:Category {name: $category_name})"
                        "MERGE (p)-[:BELONGS_TO]->(c)",
                        product_category)

        # Add SOLD_BY relationships
        for product_store in product_stores:
            session.run("MATCH (p:Product {asin: $asin})"
                        "MATCH (s:Store {name: $store_name})"
                        "MERGE (p)-[:SOLD_BY]->(s)",
                        product_store)
# Execute the graph creation
create_graph(driver, products, users, categories, stores)

# Close the driver connection when done
driver.close()

NameError: name 'users' is not defined

In [ ]:
from neo4j import GraphDatabase

# Function to find the most popular category
def find_most_popular_category(driver):
    with driver.session() as session:
        result = session.run("""
            MATCH (p:Product)-[:BELONGS_TO]->(c:Category)<-[:PURCHASED]-(:User)
            RETURN c.name AS Category, COUNT(*) AS Purchases
            ORDER BY Purchases DESC
            LIMIT 1
        """)
        return result.single()

# Function to find the most popular product
def find_most_popular_product(driver):
    with driver.session() as session:
        result = session.run("""
            MATCH (p:Product)<-[:PURCHASED]-(:User)
            RETURN p.title AS Product, COUNT(*) AS Purchases
            ORDER BY Purchases DESC
            LIMIT 1
        """)
        return result.single()

# Connect to the Neo4j database

uri = "neo4j://8.tcp.ngrok.io:17322 -> localhost:7687"
username = "neo4j"
password = "apan5400"
driver = GraphDatabase.driver(uri, auth=(username, password))

# Find and print the most popular category
most_popular_category = find_most_popular_category(driver)
if most_popular_category:
    print(f"The most popular category is: {most_popular_category['Category']} with {most_popular_category['Purchases']} purchases.")
else:
    print("No popular category found.")

# Find and print the most popular product
most_popular_product = find_most_popular_product(driver)
if most_popular_product:
    print(f"The most popular product is: {most_popular_product['Product']} with {most_popular_product['Purchases']} purchases.")
else:
    print("No popular product found.")

# Close the driver connection when done
driver.close()


No popular category found.
The most popular product is: Tyger Auto T1 Soft Roll Up Truck Bed Tonneau Cover Compatible with 2004-2012 Chevy Colorado / GMC Canyon; 2006-2008 Isuzu I280 | Fleetside 6'1" Bed (72") | TG-BC1C9002 with 1 purchases.


In [ ]:
# Function to find frequent pairs of products
def find_frequent_product_pairs(driver):
    query = """
    MATCH (u:User)-[:PURCHASED]->(p1:Product)
    WITH u, p1
    MATCH (u)-[:PURCHASED]->(p2:Product)
    WHERE p1 <> p2 AND id(p1) < id(p2)
    RETURN p1.title AS Product1, p2.title AS Product2, COUNT(*) AS Frequency
    ORDER BY Frequency DESC
    LIMIT 10
    """
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([record.data() for record in result])


# Connect to the Neo4j database
uri = "neo4j://8.tcp.ngrok.io:17322 -> localhost:7687"
username = "neo4j"
password = "apan5400"
driver = GraphDatabase.driver(uri, auth=(username, password))

# Find frequent product pairs
frequent_pairs_df = find_frequent_product_pairs(driver)
print(frequent_pairs_df)
# Close the driver connection when done
driver.close()

                                            Product1  \
0  Walker Exhaust SoundFX 18963 Direct Fit Exhaus...   
1                                            perfect   

                                            Product2  Frequency  
0  Walker Exhaust SoundFX 18962 Direct Fit Exhaus...          1  
1  AIP Electronics Premium Ignition Coil Pack Com...          1  


In [ ]:
import pandas as pd
from neo4j import GraphDatabase

# Function to find category trends
def find_category_trends(driver):
    query = """
    MATCH (p:Product)-[:BELONGS_TO]->(c:Category)<-[r:PURCHASED]-(u:User)
    WITH c.name AS Category, date(r.date) AS Date, COUNT(*) AS Purchases
    RETURN Category, Date, Purchases
    ORDER BY Category, Date
    """
    with driver.session() as session:
        result = session.run(query)
        # Convert to DataFrame
        return pd.DataFrame([record.data() for record in result])

# Connect to the Neo4j database
uri = "neo4j://8.tcp.ngrok.io:17322 -> localhost:7687"
username = "neo4j"
password = "apan5400"
driver = GraphDatabase.driver(uri, auth=(username, password))

# Get category trends
category_trends_df = find_category_trends(driver)
print(category_trends_df)

# Close the driver connection when done
driver.close()

Empty DataFrame
Columns: []
Index: []


In [ ]:
def check_connection():
    try:
        graph.run("RETURN 'Connection Successful'").data()
    except Exception as e:
        print(f"Connection failed: {e}")

check_connection()



Connection failed: Cannot connect to any known routers


In [ ]:
query = ("MATCH (n) DETACH DELETE n")
with driver.session() as session:
    session.run (query)

print("All Nodes and relationships are deleted!")

<ipython-input-70-11bb0769ea06>:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


All Nodes and relationships are deleted!


In [ ]:
def create_connection(uri, user, password):
    return GraphDatabase.driver(uri, auth=(username, password))
driver = create_connection(uri, username, password)

In [ ]:
#The plan:

#Product Nodes: Each product can be a node, with properties like title, average_rating, price, etc.
#User Nodes: Each user can also be a node, with a user_id property.
#Category Nodes: Categories can be nodes as well, with the category name as a property.
#Store Nodes: Stores are also nodes, with the store name as a property.
#PURCHASED Relationships: Connect User nodes to Product nodes to represent a purchase. The rating could be a property of this relationship.
#BELONGS_TO Relationship: Connect Product nodes to Category nodes to represent the categories each product belongs to.
#SOLD_BY Relationship: Connect Product nodes to Store nodes to represent the store selling the product.